In [ ]:
import csv
import random
goodreads_book_dict = {}
book_dict = {}
goodreads_book_id = []
book_id = []
create_books_data = []
with open('/content/books_data.csv','r') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=';')
  for row in spamreader:
    goodreads_book_id.append(row[1])
    book_id.append(row[0])
    row[5] = row[5] if row[5] else str(random.randint(100,10000000))
    create_books_data.append(row)
    goodreads_book_dict[row[1]] = row[5]
    book_dict[row[0]] = row[5]

create_books_data = create_books_data[1:]
goodreads_book_id = goodreads_book_id[1:]
book_id = book_id[1:]

In [ ]:
with open("books_queries.txt", 'w') as file:
  for row in create_books_data:
    year = row[8].split('.')[0] if row[8] else "0"
    title = row[10].replace('"', '')
    query = "CREATE (p:Book { isbn: \""+row[5]+"\", year: "+year+", title: \""+title+"\", image: \""+row[21]+"\" }) RETURN p;\n"
    file.write(query)

In [ ]:
book_tags = []
tag_id = []

with open('/content/book_tags.csv') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=',')
  for row in spamreader:
    if row[0] in goodreads_book_id:
      book_tags.append(row)
      if row[1] not in tag_id:
        tag_id.append(row[1])

In [ ]:
tags = []
with open('/content/tags.csv') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=',')
  for row in spamreader:
    if row[0] in tag_id:
      tags.append(row)

In [ ]:
tags_dict = {}
with open("tags_queries.txt", 'w') as file:
  for row in tags:
    query = "CREATE (p:Tag { name: \""+row[1]+"\" }) RETURN p;\n"
    file.write(query)
    tags_dict[row[0]] = row[1]

In [ ]:
with open("books_tags_queries.txt", 'w') as file:
  for row in book_tags:
    tag = tags_dict[row[1]]
    isbn = goodreads_book_dict[row[0]]
    query = "MATCH (t:Tag) WHERE t.name = \"" +tag+"\" MATCH (b:Book) WHERE b.isbn = \""+isbn+"\" CREATE (b)-[:IN_TAG {count: "+row[2]+" }]->(t) RETURN t, b;\n"
    file.write(query)

In [ ]:
ratings = []
users = []
book_id_reviews = {}
with open('/content/ratings.csv') as csvfile:
  spamreader = csv.reader(csvfile, delimiter=',')
  for row in spamreader:
    if row[1] in book_id:
      book_id_reviews.setdefault(row[1], []).append(1)
      if len(book_id_reviews[row[1]]) < 50:
        ratings.append(row)
        users.append(row[0])

24500


In [ ]:
users_set = set(users)
unique_users = (list(users_set))

6276


In [ ]:
!pip install names

     |████████████████████████████████| 789 kB 7.0 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803698 sha256=36d780718da20e3a1d37889bd83d88ec7590bc5ddafcb75b79f0d82cb55052c5
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names


In [ ]:
import names

users = []
users_dict = {}
for user_id in unique_users:
  username = names.get_full_name().replace(' ', '_')
  users.append([user_id, username])
  users_dict[user_id] = email

In [ ]:
with open("users_queries.txt", 'w') as file:
  for row in users:
    query = "CREATE (p:User { username: \""+row[1]+"\",  password: \""+row[2]+"\" }) RETURN p;\n"
    file.write(query)

In [ ]:
with open("users_ratings_queries.txt", 'w') as file:
  for row in ratings:
    email = users_dict[row[0]]
    isbn = book_dict[row[1]]
    query = "MATCH (u:User) WHERE u.email = \""+email+"\" MATCH (b:Book) WHERE b.isbn = \""+isbn+"\" CREATE (u)-[r:RATED {rating: "+row[2]+"}]->(b) RETURN u, b, r;\n"
    file.write(query)

In [ ]:
authors = []
for row in create_books_data:
  temp = row[7].split(',')
  for author in temp:
    authors.append(" ".join(author.split()))

authors_set = set(authors)
unique_authors = (list(authors_set))

with open("authors_queries.txt", 'w') as file:
  for author in unique_authors:
    query = "CREATE (a:Author { name: \""+author+"\" }) RETURN a;\n"
    file.write(query)

In [ ]:
with open("authors_books_queries.txt", 'w') as file:
  for row in create_books_data:
    authors = row[7]
    authors = authors.split(',')
    isbn = row[5]
    for author in authors:
      query = "MATCH (a:Author) WHERE a.name = \""+" ".join(author.split())+"\" MATCH (b:Book) WHERE b.isbn = \""+ isbn +"\" CREATE (a)-[:WROTE]->(b) RETURN a, b;\n"
      file.write(query)

In [ ]:
users_emails = list(users_dict.values())
with open("users_following_queries.txt", 'w') as file:
  for key, value in users_dict.items():
    email = value
    for _ in range(3):
      email2 = random.choice(users_emails)
      query = "MATCH (p1:User) WHERE p1.email = \""+email+"\" MATCH (p2:User) WHERE p2.email =\""+email2+"\" CREATE (p1)-[:FOLLOWS]->(p2) RETURN p1, p2;\n"
      file.write(query)